In [1]:
# notebook dependencies 
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300

import pandas as pd
import numpy as np
import os

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# time module
from time import time

from initial_acquire import get_majors_df

In [2]:
# getting the initial majors dataset 

df = get_majors_df()
df.head()

dataframe shape: (224849, 3109)


,UNITID,OPEID6_x,INSTNM_x,CONTROL_x,MAIN_x,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelors Degree,NaN,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
1,100654,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelors Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
2,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelors Degree,9.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
3,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
4,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,6,Doctoral Degree,1.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
